In [3]:
import math
from neuroml import NeuroMLDocument
from neuroml import Cell
from neuroml import BiophysicalProperties
from neuroml import MembraneProperties
from neuroml import IncludeType
from neuroml import ChannelDensity, ChannelDensityNernst, ChannelDensityNernstCa2, ChannelDensityGHK, ChannelDensityGHK2
from neuroml import SpikeThresh
from neuroml import SpecificCapacitance
from neuroml import InitMembPotential
from neuroml import IntracellularProperties
from neuroml import Resistivity, Species
from neuroml import Morphology, Segment, Point3DWithDiam
from neuroml import Network, Population
from neuroml import PulseGenerator, ExplicitInput
from pyneuroml import pynml
import numpy as np
from pyneuroml.lems import LEMSSimulation

In [9]:
def main():
    """Main function

    Include the NeuroML model into a LEMS simulation file, run it, plot some
    data.
    """
    # Simulation bits

    sim_id = "pyr_single_comp_ca"
    simulation = LEMSSimulation(sim_id=sim_id, duration=700, dt=0.005, simulation_seed=123)

    # Include the NeuroML model file
    simulation.include_neuroml2_file(create_network())
    # Assign target for the simulation
    simulation.assign_simulation_target("single_pyr_cell_network")

    # Recording information from the simulation
    simulation.create_output_file(id="output0", file_name=sim_id + ".dat")
    simulation.add_column_to_output_file("output0", column_id="pop0[0]_v", quantity="pop0[0]/v")

    # simulation.create_output_file(id="cai", file_name=sim_id + ".cai.dat")
    # simulation.add_column_to_output_file("cai", column_id="pop0[0]_cai", quantity="pop0[0]/cai")

    # Recording information from the simulation
    simulation.create_output_file(id="sca_m", file_name=sim_id + ".sca_m.dat")
    simulation.add_column_to_output_file("sca_m", column_id="pop0[0]_sca_m", quantity="pop0[0]/pyr_b_prop/membraneProperties/soma_sca/sca/m/q")

    simulation.create_output_file(id="sca_h", file_name=sim_id + ".sca_h.dat")
    simulation.add_column_to_output_file("sca_h", column_id="pop0[0]_sca_h", quantity="pop0[0]/pyr_b_prop/membraneProperties/soma_sca/sca/h/q")

    simulation.create_output_file(id="kca_n", file_name=sim_id + ".kca_n.dat")
    simulation.add_column_to_output_file("kca_n", column_id="pop0[0]_kca_n", quantity="pop0[0]/pyr_b_prop/membraneProperties/soma_kca/kca/n/q")

    # Save LEMS simulation to file
    sim_file = simulation.save_to_file()

    # Run the simulation using the default jNeuroML simulator
    pynml.run_lems_with_jneuroml(sim_file, max_memory="2G", nogui=True, plot=False)
    # Plot the data
    plot_data(sim_id)

In [5]:
def plot_data(sim_id):
    """Plot the sim data.

    Load the data from the file and plot the graph for the membrane potential
    using the pynml generate_plot utility function.

    :sim_id: ID of simulaton

    """
    data_array = np.loadtxt(sim_id + ".dat")
    pynml.generate_plot([data_array[:, 0]], [data_array[:, 1]], "Membrane potential", show_plot_already=False, save_figure_to=sim_id + "-v.png", xaxis="time (s)", yaxis="membrane potential (V)")

    data_array_sca_m = np.loadtxt(sim_id + ".sca_m.dat")
    pynml.generate_plot([data_array_sca_m[:, 0]], [data_array_sca_m[:, 1]], "sca m gate", show_plot_already=False, save_figure_to=sim_id + "-sca_m.png", xaxis="time (s)", yaxis="q")

    data_array_sca_h = np.loadtxt(sim_id + ".sca_h.dat")
    pynml.generate_plot([data_array_sca_h[:, 0]], [data_array_sca_h[:, 1]], "sca h gate", show_plot_already=False, save_figure_to=sim_id + "-sca_h.png", xaxis="time (s)", yaxis="q")

    data_array_kca_n = np.loadtxt(sim_id + ".kca_n.dat")
    pynml.generate_plot([data_array_kca_n[:, 0]], [data_array_kca_n[:, 1]], "kca n gate", show_plot_already=False, save_figure_to=sim_id + "-kca_n.png", xaxis="time (s)", yaxis="q")


In [6]:
def create_cell():
    pyr_cell_doc = NeuroMLDocument(id='cell', notes="Layer 5 Pyramidal cell")
    pyr_cell_fn = "pyr_ca_cell.nml"
 
    pyr_cell_doc.includes.append(IncludeType("../pas.channel.nml"))
    # pyr_cell_doc.includes.append(IncludeType("nat.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("../cad.nml"))
    pyr_cell_doc.includes.append(IncludeType("../sca.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("../kca.channel.nml"))

    # Define a cell
    pyr_cell = Cell(id="pyr_cell", notes="A single compartment Layer 5 Pyramidal cell")

    # Define its biophysical properties
    bio_prop = BiophysicalProperties(id="pyr_b_prop")
    #  notes="Biophysical properties for Layer 5 Pyramidal cell")

    # Membrane properties are a type of biophysical properties
    mem_prop = MembraneProperties()
    # Add membrane properties to the biophysical properties
    bio_prop.membrane_properties = mem_prop

    # Append to cell
    pyr_cell.biophysical_properties = bio_prop


    pas_channel_density = ChannelDensity(id="pas_channels", cond_density="0.485726 S_per_m2", erev="-80.3987 mV", ion="non_specific", ion_channel="pas")
    mem_prop.channel_densities.append(pas_channel_density)

    # nat_channel_density = ChannelDensity(id="nat_channels", cond_density="236.616175 S_per_m2", erev="55 mV", ion="na", ion_channel="nat")
    # mem_prop.channel_densities.append(nat_channel_density)

    kca_channel_density = ChannelDensityGHK2(id="soma_kca", ion_channel="kca", cond_density="6.15058501 S_per_m2", ion="k")
    mem_prop.channel_density_ghk2s.append(kca_channel_density)

    # sca_soma
    sca_channel_density = ChannelDensityGHK2(id="soma_sca", ion_channel="sca", cond_density="0.454235280 S_per_m2", ion="ca")
    mem_prop.channel_density_ghk2s.append(sca_channel_density)

    # Other membrane properties
    mem_prop.spike_threshes.append(SpikeThresh(value="-20mV"))
    mem_prop.specific_capacitances.append(SpecificCapacitance(value="2.23041 uF_per_cm2"))
    mem_prop.init_memb_potentials.append(InitMembPotential(value="-65mV"))

    intra_prop = IntracellularProperties()
    intra_prop.resistivities.append(Resistivity(value="0.082 kohm_cm"))

    intra_prop.species.append(Species(id="ca", concentration_model="cad", ion="ca", initial_concentration="100e-6 mM",
                             initial_ext_concentration="2 mM"))

    # Add to biological properties
    bio_prop.intracellular_properties = intra_prop

    # Morphology
    morph = Morphology(id="pyr_cell_morph")
    #  notes="Simple morphology for the HH cell")
    seg = Segment(id="0", name="soma", notes="Soma segment")
    # We want a diameter such that area is 1000 micro meter^2
    # surface area of a sphere is 4pi r^2 = 4pi diam^2
    # diam = math.sqrt(1682 / math.pi)
    proximal = Point3DWithDiam(x="0", y="0", z="0", diameter=str(23.1453))
    distal = Point3DWithDiam(x="0", y="23.1453", z="0", diameter=str(23.1453))
    seg.proximal = proximal
    seg.distal = distal
    morph.segments.append(seg)
    pyr_cell.morphology = morph

    pyr_cell_doc.cells.append(pyr_cell)
    pynml.write_neuroml2_file(nml2_doc=pyr_cell_doc, nml2_file_name=pyr_cell_fn, validate=True)
    return pyr_cell_fn

create_cell()

pyNeuroML >>> INFO - Executing: (java -Xmx400M  -jar  "/Users/shayan/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/pyneuroml/lib/jNeuroML-0.12.0-jar-with-dependencies.jar" -validate "pyr_ca_cell.nml" ) in directory: .
pyNeuroML >>> INFO - Command completed. Output: 
  jNeuroML >>   jNeuroML v0.12.0
  jNeuroML >>  Validating: /Users/shayan/Desktop/OSB/BahlEtAl2012_ReducedL5PyrCell/NeuroML2/test_ca/pyr_ca_cell.nml
  jNeuroML >>  Valid against schema and all tests
  jNeuroML >>  No warnings
  jNeuroML >>  
  jNeuroML >>  Validated 1 files: All valid and no warnings
  jNeuroML >>  
  jNeuroML >>  


'pyr_ca_cell.nml'

In [7]:
def create_network():
    """Create the network

    :returns: name of network nml file
    """
    net_doc = NeuroMLDocument(id="network",
                              notes="Pyramidal cell network")
    net_doc_fn = "pyr_example_ca_net.nml"
    net_doc.includes.append(IncludeType(href=create_cell()))
    # Create a population: convenient to create many cells of the same type
    pop = Population(id="pop0", notes="A population for pyramidal cell", component="pyr_cell", size=1)
    # Input
    pulsegen = PulseGenerator(id="pg", notes="Simple pulse generator", delay="100ms", duration="500ms", amplitude="0.1nA")

    exp_input = ExplicitInput(target="pop0[0]", input="pg")

    net = Network(id="single_pyr_cell_network",
                  type="networkWithTemperature",
                  temperature = "37 degC",
                  note="A network with a single population")
    net_doc.pulse_generators.append(pulsegen)
    net.explicit_inputs.append(exp_input)
    net.populations.append(pop)
    net_doc.networks.append(net)

    pynml.write_neuroml2_file(nml2_doc=net_doc, nml2_file_name=net_doc_fn, validate=True)
    return net_doc_fn

In [10]:
if __name__ == "__main__":
    main()

pyNeuroML >>> INFO - Executing: (java -Xmx400M  -jar  "/Users/shayan/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/pyneuroml/lib/jNeuroML-0.12.0-jar-with-dependencies.jar" -validate "pyr_ca_cell.nml" ) in directory: .
pyNeuroML >>> INFO - Command completed. Output: 
  jNeuroML >>   jNeuroML v0.12.0
  jNeuroML >>  Validating: /Users/shayan/Desktop/OSB/BahlEtAl2012_ReducedL5PyrCell/NeuroML2/test_ca/pyr_ca_cell.nml
  jNeuroML >>  Valid against schema and all tests
  jNeuroML >>  No warnings
  jNeuroML >>  
  jNeuroML >>  Validated 1 files: All valid and no warnings
  jNeuroML >>  
  jNeuroML >>  
pyNeuroML >>> INFO - Executing: (java -Xmx400M  -jar  "/Users/shayan/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/pyneuroml/lib/jNeuroML-0.12.0-jar-with-dependencies.jar" -validate "pyr_example_ca_net.nml" ) in directory: .
pyNeuroML >>> INFO - Command completed. Output: 
  jNeuroML >>   jNeuroML v0.12.0
  jNeuroML >>  Validating: /Users/shayan/Desktop/OSB/BahlEtAl2012_ReducedL

SystemExit: 1

/Users/shayan/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
